# Boilerplate

In [ ]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper

init_logging()

# Run

List all Python wrappers

In [ ]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

## Transport layer setup

Create the Transport layer and run it in the background.

In [ ]:
tiface = await new_transport_interface("0.0.0.0", 12345, TransportModeW("client"))
tiface.run()

# Filter layer setup

In [ ]:
fiface = FilterInterface(tiface, FilterModeW("client"))

Find methods to run

In [ ]:
[m for m in dir(fiface) if not m.startswith('__')]

Run Filter!

In [ ]:
run_fut = fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not run_fut.done()

Get the number of milliseconds between calling `get_notifications` on the contained `TransportInterface`. This can also be assigned to at any time. Lower values can cause performance issues, while higher values add latency for anything involving filter notifications.

## Filter layer operation

### As a client, connect to server

In [ ]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

In [ ]:
chococat_endpoint = EndpointW("157.230.134.224:2016")
filter_cmd = FilterCmdW("sendrequestaction", endpoint=chococat_endpoint, action=connect_action)
filter_cmd

In [ ]:
await fiface.command_response(filter_cmd)

In [ ]:
time.sleep(1)
fiface.get_notifications()

In [ ]:
new_response_code = _[0]
new_response_code

In [ ]:
cookie = new_response_code.response_code.cookie
cookie

### Convenience function

In [ ]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await fiface.command_response(filter_cmd)

### Join a room

In [ ]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [ ]:
fiface.get_notifications()